In [5]:
!pip install geoviews

   ---------------------------------------- 0.0/511.2 kB ? eta -:--:--
    --------------------------------------- 10.2/511.2 kB ? eta -:--:--
   ---------------- ----------------------- 204.8/511.2 kB 4.1 MB/s eta 0:00:01
   ---------------------------------------- 511.2/511.2 kB 5.3 MB/s eta 0:00:00


In [2]:
# Dependencies and Setup
import hvplot.pandas
import pandas as pd
import requests
#import holoviews as hv
#from holoviews import opts
!pip install cartopy
# Import API key
from api_keys import geoapify_key

In [3]:
# Load the CSV file created in Part 1 into a Pandas DataFrame
city_data_df = pd.read_csv("../output_data/cities.csv")

# Display sample data
city_data_df.head()

,City_ID,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
0,0,valparaiso,-33.0393,-71.6273,22.22,60,0,3.09,CL,1704927815
1,1,flying fish cove,-10.4217,105.6791,23.82,94,40,1.03,CX,1704927933
2,2,edinburgh of the seven seas,-37.0676,-12.3116,16.31,85,98,2.70,SH,1704928130
3,3,jayapura,-2.5333,140.7000,27.66,85,99,1.59,ID,1704928131
4,4,adamstown,-25.0660,-130.1015,24.69,81,66,4.24,PN,1704927840


---

### Step 1: Create a map that displays a point for every city in the `city_data_df` DataFrame. The size of the point should be the humidity in each city.

In [6]:
%%capture --no-display

# Configure the map plot

map_plot = city_data_df.hvplot.points(
    "Lng",
    "Lat",
    geo=True,
    tiles = "OSM",
    frame_width = 800,
    frame_height = 600,
    size = "Humidity",
    scale = 0.9,
    color = "City"
)

# Display the map

map_plot

:Overlay
   .WMTS.I   :WMTS   [Longitude,Latitude]
   .Points.I :Points   [Lng,Lat]   (City,Humidity)

### Step 2: Narrow down the `city_data_df` DataFrame to find your ideal weather condition

In [10]:
# Narrow down cities that fit criteria and drop any results with null values

city_ideal_df = city_data_df.loc[
    (city_data_df["Max Temp"] >21) \
    & (city_data_df["Max Temp"] < 27) \
    & (city_data_df["Wind Speed"] < 4.5) \
    & (city_data_df["Cloudiness"] == 0)
]

# Drop any rows with null values
city_ideal_df = city_ideal_df.dropna()

# Display sample data
city_ideal_df

,City_ID,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
0,0,valparaiso,-33.0393,-71.6273,22.22,60,0,3.09,CL,1704927815
133,133,ukata,10.8312,5.8249,22.38,11,0,3.57,NG,1704928148
175,175,nova sintra,14.8667,-24.7167,21.18,66,0,4.37,CV,1704928153
206,206,tazacorte,28.6290,-17.9293,21.75,35,0,3.13,ES,1704928158
472,472,ormara,25.2088,64.6357,21.11,45,0,1.75,PK,1704928192
510,510,goz beida,13.9500,20.5500,21.62,24,0,3.21,TD,1704928197


### Step 3: Create a new DataFrame called `hotel_df`.

In [11]:
# Use the Pandas copy function to create DataFrame called hotel_df to store the city, country, coordinates, and humidity
hotel_df = city_ideal_df[["City","Country","Lat","Lng","Humidity"]].copy()

# Add an empty column, "Hotel Name," to the DataFrame so you can store the hotel found using the Geoapify API
hotel_df.loc[:,"Hotel Name"] = ""

# Display sample data
hotel_df

,City,Country,Lat,Lng,Humidity,Hotel Name
0,valparaiso,CL,-33.0393,-71.6273,60,
133,ukata,NG,10.8312,5.8249,11,
175,nova sintra,CV,14.8667,-24.7167,66,
206,tazacorte,ES,28.6290,-17.9293,35,
472,ormara,PK,25.2088,64.6357,45,
510,goz beida,TD,13.9500,20.5500,24,


### Step 4: For each city, use the Geoapify API to find the first hotel located within 10,000 metres of your coordinates.

In [12]:
# Set parameters to search for a hotel
radius = 10000
categories = "accommodation.hotel"

params = {
     "categories":categories,
    "apiKey":geoapify_key
  
}

# Print a message to follow up the hotel search
print("Starting hotel search")

# Iterate through the hotel_df DataFrame
for index, row in hotel_df.iterrows():
    # get latitude, longitude from the DataFrame
    latitude = row["Lat"]
    longitude = row["Lng"]
    
    # Add filter and bias parameters with the current city's latitude and longitude to the params dictionary
    params["filter"] = f"circle:{longitude},{latitude},{radius}"
    params["bias"] = f"proximity:{longitude},{latitude}"
    
    # Set base URL
    base_url = "https://api.geoapify.com/v2/places"


    # Make and API request using the params dictionaty
    name_address = requests.get(base_url, params=params)
    
    # Convert the API response to JSON format
    name_address = name_address = name_address.json()
    
    # Grab the first hotel from the results and store the name in the hotel_df DataFrame
    try:
        hotel_df.loc[index, "Hotel Name"] = name_address["features"][0]["properties"]["name"]
    except (KeyError, IndexError):
        # If no hotel is found, set the hotel name as "No hotel found".
        hotel_df.loc[index, "Hotel Name"] = "No hotel found"
        
    # Log the search results
    print(f"{hotel_df.loc[index, 'City']} - nearest hotel: {hotel_df.loc[index, 'Hotel Name']}")

# Display sample data
hotel_df

Starting hotel search
valparaiso - nearest hotel: Ibis Hotel
ukata - nearest hotel: No hotel found
nova sintra - nearest hotel: Residência Ka Dencho
tazacorte - nearest hotel: App Leyma
ormara - nearest hotel: No hotel found
goz beida - nearest hotel: No hotel found


,City,Country,Lat,Lng,Humidity,Hotel Name
0,valparaiso,CL,-33.0393,-71.6273,60,Ibis Hotel
133,ukata,NG,10.8312,5.8249,11,No hotel found
175,nova sintra,CV,14.8667,-24.7167,66,Residência Ka Dencho
206,tazacorte,ES,28.6290,-17.9293,35,App Leyma
472,ormara,PK,25.2088,64.6357,45,No hotel found
510,goz beida,TD,13.9500,20.5500,24,No hotel found


### Step 5: Add the hotel name and the country as additional information in the hover message for each city in the map.

In [13]:
%%capture --no-display

# Configure the map plot
map_plot = hotel_df.hvplot.points(
    "Lng",
    "Lat",
    geo=True,
    tiles = "OSM",
    frame_width = 800,
    frame_height = 600,
    color = "Hotel Name",
    hover_cols = ["City", "Humidity","Hotel Name", "Country"]
)
# Display the map
map_plot

:Overlay
   .WMTS.I   :WMTS   [Longitude,Latitude]
   .Points.I :Points   [Lng,Lat]   (Hotel Name,City,Humidity,Country)